# Shark League Score
## Import Python libraries

In [2]:
import pandas as pd
import numpy as np
import logging
import swc_simple_client as swc

## Configure logging

In [3]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='shark_notebook.log', level=logging.INFO,)

## Setup notebook variables

In [5]:
base_url = "https://super-duper-palm-tree-g4v9vrvxx9vgc9jqv-8000.app.github.dev"

## Get Max Scores
### USe endpoint: LIST_WEEKS_ENDPOINT

In [8]:
week_api_response = swc.call_api_endpoint(base_url, swc.LIST_WEEKS_ENDPOINT)
print(f"Response status code: {week_api_response.status_code}")
print(f"Response text: {week_api_response.text}")
print(f"Response headers: {week_api_response.headers}")

if week_api_response.status_code == 200:
    weeks_df = pd.DataFrame(week_api_response.json())
    weeks_df['year'] = weeks_df['week_number'].str.slice(0,4).astype(int)

    max_totals_grouped_df = weeks_df.groupby('year').agg(
        ppr_12_max_points=('ppr_12_total_points', 'sum'),
        half_ppr_8_max_points=('half_ppr_8_total_points', 'sum'))

    display(max_totals_grouped_df)
    logging.info("Max total points per scoring format per year calculated.")
else:
    print(f"API Error: {week_api_response.status_code}")


Response status code: 302
Response text: API error
Response headers: Headers({'content-length': '9'})
API Error: 302
